In [2]:
import tensorflow_datasets as tfds
import tensorflow as tf
from tensorflow.keras import layers, models
import keras_tuner as kt

# Load the MNIST dataset
dataset, info = tfds.load('mnist', with_info=True, as_supervised=True)

2024-07-07 06:44:24.132521: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [3]:
# Split the dataset into training and testing sets
train_dataset, test_dataset = dataset['train'], dataset['test']

In [4]:
# Function to preprocess the data
def preprocess(image, label):
    image = tf.cast(image, tf.float32) / 255.0
    return image, label

In [5]:
# Apply the preprocess function to the dataset
train_dataset = train_dataset.map(preprocess).batch(32)
test_dataset = test_dataset.map(preprocess).batch(32)

In [6]:
# Step 3: Model Selection
def build_model(hp):
    model = models.Sequential()
    model.add(layers.Flatten(input_shape=(28, 28)))
    model.add(layers.Dense(hp.Int('units', min_value=32, max_value=512, step=32), activation='relu'))
    model.add(layers.Dense(10, activation='softmax'))
    model.compile(optimizer='adam',
                  loss='sparse_categorical_crossentropy',
                  metrics=['accuracy'])
    return model

In [7]:
# Step 6: Model Optimization using Keras Tuner
tuner = kt.Hyperband(build_model,
                     objective='val_accuracy',
                     max_epochs=10,
                     factor=3,
                     directory='my_dir',
                     project_name='mnist')

/opt/anaconda3/lib/python3.11/site-packages/keras/src/layers/reshaping/flatten.py:37: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


In [8]:
# Early stopping callback
stop_early = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=5)

In [9]:
# Perform hyperparameter tuning
tuner.search(train_dataset, epochs=50, validation_data=test_dataset, callbacks=[stop_early])

Trial 22 Complete [00h 01m 09s]
val_accuracy: 0.9797000288963318

Best val_accuracy So Far: 0.9800999760627747
Total elapsed time: 00h 37m 20s


In [11]:
# Get the optimal hyperparameters
best_hps = tuner.get_best_hyperparameters(num_trials=1)[0]

In [12]:
# Build the model with the optimal hyperparameters
model = build_model(best_hps)

In [13]:
# Step 4: Model Training with optimal hyperparameters
model.fit(train_dataset, epochs=5)

Epoch 1/5
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 18s 9ms/step - accuracy: 0.9001 - loss: 0.3354
Epoch 2/5
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 18s 9ms/step - accuracy: 0.9725 - loss: 0.0895
Epoch 3/5
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 23s 11ms/step - accuracy: 0.9841 - loss: 0.0556
Epoch 4/5
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 21s 11ms/step - accuracy: 0.9891 - loss: 0.0363
Epoch 5/5
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 19s 10ms/step - accuracy: 0.9934 - loss: 0.0234


In [12]:
# Step 5: Model Evaluation
test_loss, test_acc = model.evaluate(test_dataset)
print(f'Test Accuracy: {test_acc}')

313/313 ━━━━━━━━━━━━━━━━━━━━ 2s 2ms/step - accuracy: 0.9744 - loss: 0.0834
Test Accuracy: 0.9739999771118164


In [13]:
# Step 7: Model Testing (Evaluating the optimized model)
print("Testing the optimized model:")
test_loss, test_acc = model.evaluate(test_dataset)
print(f'Optimized Test Accuracy: {test_acc}')

Testing the optimized model:
313/313 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.9744 - loss: 0.0834
Optimized Test Accuracy: 0.9739999771118164
